In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
import os
import numpy as np
import pandas as pd

In [6]:
names = [
 'music',
 'brother',
 'ocean',
 'i love you',
 'dog',
 'polite',
 'milk',
 'money',
 'coffee',
 'door']

In [7]:
len(names)

10

In [8]:
import glob

df = pd.DataFrame()
for label in names:
    files = glob.glob(f"/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/mediapipe/reformatting-the-data/data_25_labels/{label}/*.csv")
    for f in files:
        csv = pd.read_csv(f)
        df = pd.concat([df, csv])

print(df)

    class        x1        y1        z1        v1        x2        y2  \
0   music  0.510927  0.266515 -0.612352  0.999522  0.521480  0.245207   
1   music  0.510671  0.267104 -0.641056  0.999552  0.521454  0.245630   
2   music  0.510515  0.267290 -0.646706  0.999580  0.521436  0.245851   
3   music  0.510243  0.267347 -0.649374  0.999607  0.521376  0.245964   
4   music  0.510031  0.267435 -0.646747  0.999630  0.521279  0.246073   
..    ...       ...       ...       ...       ...       ...       ...   
62   door  0.524986  0.247936 -0.857997  0.997421  0.539278  0.204997   
63   door  0.525036  0.246169 -0.849091  0.995625  0.539338  0.203739   
64   door  0.525020  0.243054 -0.829668  0.993639  0.539431  0.201486   
65   door  0.524460  0.240872 -0.891451  0.992904  0.539293  0.199475   
66   door  0.523154  0.238548 -0.873610  0.988032  0.539075  0.197419   

          z2        v2        x3  ...  z73  v73  x74  y74  z74  v74  x75  y75  \
0  -0.579605  0.999107  0.527378  ...  0.0

In [9]:
df_features = df.copy()

df_labels = df_features.pop("class")

df_features = np.array(df_features)

print(df_features.shape)
print(df_labels.shape)
df_labels.head()

(24430, 300)
(24430,)


0    music
1    music
2    music
3    music
4    music
Name: class, dtype: object

In [10]:
dicti = {names[i]: i for i in range(len(names))}
dicti

{'music': 0,
 'brother': 1,
 'ocean': 2,
 'i love you': 3,
 'dog': 4,
 'polite': 5,
 'milk': 6,
 'money': 7,
 'coffee': 8,
 'door': 9}

In [11]:
def change_label(x):
    return dicti[x]

df_labels = df_labels.apply(change_label)
df_labels.head()

0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

In [12]:
df_labels = np.array([df_labels]).T


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_features, df_labels, test_size=0.2)
print(X_train.shape)
print(X_train)
print(y_train.shape)
print(y_train)

(19544, 300)
[[ 0.33048332  0.26185474 -0.41955152 ...  0.48287541 -0.00866116
   0.        ]
 [ 0.51163518  0.29952836 -0.2786167  ...  0.          0.
   0.        ]
 [ 0.30530292  0.35854673 -0.54659903 ...  0.85765427 -0.03312659
   0.        ]
 ...
 [ 0.48500836  0.30808356 -0.34180525 ...  0.505153   -0.01313204
   0.        ]
 [ 0.49011928  0.32622653 -0.61105752 ...  0.          0.
   0.        ]
 [ 0.49476928  0.31737584 -0.3272993  ...  0.70841032 -0.00881404
   0.        ]]
(19544, 1)
[[8]
 [0]
 [7]
 ...
 [1]
 [6]
 [1]]


In [14]:
model = Sequential([
    #if it overfits, I can use dropout layers (it only gets some of the inputs from the previous, so you wanna )
    Dense(units=256, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=10, activation='softmax')
])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-23 22:54:39.904438: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-23 22:54:39.904644: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [16]:
history = model.fit(X_train,y_train, epochs=10, validation_split=0.1)

Epoch 1/10


2023-03-23 22:54:47.454940: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-23 22:54:47.831589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


550/550 [==============================] - ETA: 0s - loss: 1.7302 - accuracy: 0.3784

2023-03-23 22:54:52.446230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


550/550 [==============================] - 5s 8ms/step - loss: 1.7302 - accuracy: 0.3784 - val_loss: 1.4705 - val_accuracy: 0.4803
Epoch 2/10
550/550 [==============================] - 4s 8ms/step - loss: 1.3146 - accuracy: 0.5404 - val_loss: 1.1975 - val_accuracy: 0.5974
Epoch 3/10
550/550 [==============================] - 4s 6ms/step - loss: 1.0910 - accuracy: 0.6270 - val_loss: 1.0054 - val_accuracy: 0.6665
Epoch 4/10
550/550 [==============================] - 4s 8ms/step - loss: 0.9402 - accuracy: 0.6818 - val_loss: 0.9001 - val_accuracy: 0.7043
Epoch 5/10
550/550 [==============================] - 3s 6ms/step - loss: 0.8332 - accuracy: 0.7203 - val_loss: 0.8009 - val_accuracy: 0.7146
Epoch 6/10
550/550 [==============================] - 3s 6ms/step - loss: 0.7438 - accuracy: 0.7529 - val_loss: 0.7236 - val_accuracy: 0.7724
Epoch 7/10
550/550 [==============================] - 3s 6ms/step - loss: 0.6766 - accuracy: 0.7759 - val_loss: 0.7007 - val_accuracy: 0.7678
Epoch 8/10
550/55

In [17]:
model.save("saved_model_10_labels")

INFO:tensorflow:Assets written to: saved_model_10_labels/assets
